# Basics with Perceptron (P)

* Find this notebook at `EpyNN/epynnlive/dummy_boolean/train.ipynb`.
* Regular python code at `EpyNN/epynnlive/dummy_boolean/train.py`.

Run the notebook online with [Google Colab](https://colab.research.google.com/github/Synthaze/EpyNN/blob/main/epynnlive/dummy_boolean/train.ipynb).

**Level: Beginner**

In this notebook we will review:

* Handling Boolean data.
* Designing and training a simple perceptron using EpyNN objects.
* Basics and general concepts relevant to the context.

**<font color='red'>This notebook does not enhance, extend or replace EpyNN's documentation.</font>**

**Relevant documentation pages for the current notebook:**

* [Neural Network - Model](https://epynn.net/EpyNN_Model.html)
* [Architecture layers - Model](https://epynn.net/Layer_Model.html)
* [Data - Model](https://epynn.net/Data_Model.html)
* [Data Embedding (Input)](https://epynn.net/Embedding.html)
* [Fully Connected (Dense)](https://epynn.net/Dense.html)

## Import, configure and retrieve data

Follow [this link](prepare_dataset.ipynb) for details about data preparation.

We will import all libraries and configure seeding, behaviors and directory. 

### Imports

In [1]:
# EpyNN/epynnlive/dummy_boolean/train.ipynb
# Install dependencies
!pip3 install --upgrade-strategy only-if-needed epynn

# Standard library imports
import random

# Related third party imports
import numpy as np

# Local application/library specific imports
import epynn.initialize
from epynn.commons.library import (
    configure_directory,
    read_model,
)
from epynn.network.models import EpyNN
from epynn.embedding.models import Embedding
from epynn.dense.models import Dense
from epynnlive.dummy_boolean.prepare_dataset import prepare_dataset


Note that we imported all libraries we will use, at once and on top of the script. Even though we are going through a notebook, we should pay attention to follow good practices for imports as stated in [PEP 8 -- Style Guide for Python Code](https://www.python.org/dev/peps/pep-0008/#imports).

You may have also noted that ``# Related third party imports`` are limited to ``numpy``. 

We developed an educational resource for which computations rely on pure Python/NumPy, nothing else.

### Configuration

Let's now proceed with the configuration and preferences for the current script.

In [2]:
random.seed(1)

np.set_printoptions(threshold=10)

np.seterr(all='warn')

configure_directory(clear=True)    # This is a dummy example

Make: /pylibs/EpyNN/epynnlive/dummy_boolean/datasets
Make: /pylibs/EpyNN/epynnlive/dummy_boolean/models
Make: /pylibs/EpyNN/epynnlive/dummy_boolean/plots


We already explained the [reason for seeding](prepare_dataset.ipynb#Seeding).

The call to ``np.set_printoptions()`` set the printing behavior of NumPy arrays. To not overfill this notebook, we instructed that beyond ``threshold=10`` NumPy will trigger summarization rather than full representation.

The call to ``np.seterr()`` is **very important** if you want to be aware of what's happening in your Network. Follow the [numpy.seterr](https://numpy.org/doc/stable/reference/generated/numpy.seterr.html) official documentation for details. Herein, we make sure that floating-point errors will always ``warn`` on the terminal session and thus we will always be aware of them.

Finally, the call to ``configure_directory()`` is purely facultative but creates the default EpyNN subdirectories in the working directory.

### Retrieve Boolean features and label 

From [prepare_dataset](prepare_dataset.ipynb#Preparedataset) we imported the function ``prepare_dataset()``.

In [3]:
X_features, Y_label = prepare_dataset(N_SAMPLES=50)

TypeError: 'module' object is not callable

Let's inspect, as always.

In [ ]:
for sample in list(zip(X_features, Y_label))[:5]:
    features, label = sample
    print(label, (features.count(True) > features.count(False)), features)

We have what we expect. Remember that the conditional expression is the dummy law we used to assign a label to dummy sample Boolean features.

## Perceptron - Single layer Neural Network

Herein, we are going to build the most simple Neural Network and train it in the most simple way we can.

### The Embedding layer object

In EpyNN, data must be passed as arguments upon call of the ``Embedding()`` layer class constructor.

The instantiated object - the embedding or input layer - is always the first layer in Neural Networks made with EpyNN.

In [ ]:
embedding = Embedding(X_data=X_features,
                      Y_data=Y_label,
                      relative_size=(2, 1, 0)) # Training, validation, testing set

The arguments ``X_features`` and ``Y_label`` passed in the class constructor have been split with respect to ``relative_size`` for training, validation and testing sets.

Let’s take a look at what ``relative_size`` means.

In [ ]:
dataset = list(zip(X_features, Y_label))   # Pair-wise X-Y data

# We print the 10 first only just to not overfill the notebook
for features, label in dataset[:10]:
    print(features, label)

Now, we can apply ``relative_size`` to split the whole dataset into parts.

In [ ]:
relative_size=(2, 1, 0)

dtrain_relative, dval_relative, dtest_relative = relative_size

# Compute absolute sizes with respect to full dataset
sum_relative = sum([dtrain_relative, dval_relative, dtest_relative])

dtrain_length = round(dtrain_relative / sum_relative * len(dataset))
dval_length = round(dval_relative / sum_relative * len(dataset))
dtest_length = round(dtest_relative / sum_relative * len(dataset))

# Slice full dataset
dtrain = dataset[:dtrain_length]
dval = dataset[dtrain_length:dtrain_length + dval_length]
dtest = dataset[dtrain_length + dval_length:]

This is the procedure. Let’s see what happened.

In [ ]:
print(len(dataset))

print(len(dtrain), dtrain_relative)
print(len(dval), dval_relative)
print(len(dtest), dtest_relative)

With ``relative_size=(2, 1, 0)`` we asked to prepare a training set ``dtrain`` two times bigger than the validation set ``dval``.

We have 50 samples.

In [ ]:
print(50 / 3 * 2)
print(50 / 3 * 1)

In [ ]:
print(round(50 / 3 * 2))
print(round(50 / 3 * 1))

print((round(50 / 3 * 2)) == len(dtrain))
print((round(50 / 3 * 1)) == len(dval))

Note that since we have set ``relative_size=(2, 1, 0)`` the testing set is empty.

Let's explore some properties and attributes of the instantiated ``embedding`` layer object.

In [ ]:
# Type of object
print(type(embedding))

# Attributes and values of embedding layer
for i, (attr, value) in enumerate(vars(embedding).items()):
    print(i, attr, value)

**Lines 0-9**: Inherited from ``epynn.commons.models.Layer`` which is the [Base Layer](https://epynn.net/Layer_Model.html#base-layer). These instance attributes exist for any layers in EpyNN.

**Lines 10-15**: Instance attributes specific to ``epynn.embedding.models.Embedding`` layer. 

* (10) se_dataset: Contains data-related settings applied upon layer instantiation
* (11-13) dtrain, dval, dtest: Training, validation and testing sets in EpyNN ``dataSet`` object.
* (14) batch_dtrain: Training mini-batches. Contains the data actually used for training.
* (15) dsets: Contains the active datasets that will be evaluated during training. It contains only two ``dataSet`` objects because dval was set to empty.

### The dataSet object

Let's examine one ``dataSet`` object the same way.

In [ ]:
# Type of object
print(type(embedding.dtrain))

# Attributes and values of embedding layer
for i, (attr, value) in enumerate(vars(embedding.dtrain).items()):
    print(i, attr, value)

* (0) name: Self-explaining
* (1) active: If it contains data
* (2) X: Set of sample features
* (3) Y: Set of sample label.
* (4) y: Set of single-digit sample label.
* (5) b: Balance of labels in set.
* (6) ids: Sample identifiers.
* (7) A: Output of forward propagation
* (8) P: Label predictions.

For full documentation of the ``epynn.commons.models.dataSet`` object, you can refer to [Data - Model](https://epynn.net/Data_Model.html).

Note that in the present example we use single-digit labels and therefore the only difference between (3) and (4) is the shape. The reason for this apparent duplicate will appear in a subsequent notebook.

In [ ]:
print(embedding.dtrain.Y.shape)
print(embedding.dtrain.y.shape)

print(all(embedding.dtrain.Y.flatten() == embedding.dtrain.y))

We described how to instantiate the embedding - or input - layer with EpyNN and we browsed the attached attribute-value pairs.

We observed that it contains ``dataSet`` objects and we browsed the corresponding attribute-value pairs for the training set.

We introduced all we needed to know.

We are ready.

For more information about these concepts, please follow this link:

* [Data - Model](https://epynn.net/Data_Model.html)

### The Dense layer object

In [ ]:
dense = Dense()    # Defaults to Dense(nodes=1, activation=sigmoid)

This one was easy. Let's inspect.

In [ ]:
# Type of object
print(type(dense))

# Attributes and values of dense layer
for i, (attr, value) in enumerate(vars(dense).items()):
    print(i, attr, value)

Note that (1) indicates the number of nodes in the Dense layer, and (8, 11) the activation function for this layer. See [Activation - Functions](https://epynn.net/activation.html) for details.

For code, maths and pictures behind layers in general and the dense layer in particular, follow these links:

* [Base layer](https://epynn.net/Layer_Model.html#base-layer)
* [Template layer](https://epynn.net/Layer_Model.html#template-layer)
* [Fully Connected (Dense)](https://epynn.net/Dense.html)

## The EpyNN Network object

### Instantiate your Perceptron

Now that we have an embedding (input) layer and a Dense (output) layer, we can instantiate the EpyNN object which represents the Neural Network.

In [ ]:
layers = [embedding, dense]

The list ``layers`` is the architecture of a Perceptron.

In [ ]:
model = EpyNN(layers=layers, name='Perceptron_Dense-1-sigmoid')

The object ``EpyNN`` is the Perceptron itself.

Let's prove it.

In [ ]:
# Type of object
print(type(model))

# Attributes and values of EpyNN model for a Perceptron
for i, (attr, value) in enumerate(vars(model).items()):
    print(i, attr, value)

Does it really contain the layers we instantiated before?

In [ ]:
print((model.embedding == embedding))
print((model.layers[-1] == dense))

### Perceptron training

It does seem so, yes.

We are going to start the training of this Perceptron with all defaults, the very most simple form.

In [ ]:
model.train(epochs=100)

With all defaults, EpyNN returns:

* **EpyNN Check**: Result of a blank epoch to make sure the network is functional. For each layer, output shapes are returned for both the forward and backward propagation.
* **init_logs**: Extended report about datasets, network architecture and shapes.
* **Evaluation**: Real time evaluation of non-empty datasets (may include dtrain, dval, dtest) with default metrics accuracy and default loss function MSE.

See [How to use EpyNN - Console](https://epynn.net/quickstart.html#console) for more details.

You may also like to review the [Loss - Functions](https://epynn.net/loss.html).

For **Evaluation** and to be straightforward, we want:

* Metrics (e.g., accuracy) as high as possible.
* Cost (e.g., MSE) as low as possible.
* Differences in metrics and cost between training and validation data to be **as low as possible**, otherwise we talk about **overfitting**.

**Overfitting** happens when the model corresponds too closely to a particular set of data.

The terminal report indicates:

* Accuracy is 1 or 100% for the training set and 0.824 or 82.4% for the validation set.
* MSE is 0.058 and 0.137 for the training and validation set, respectively.
* Differences between training and validation data are significant but the model could reproduce the validation data with acceptable accuracy. Still, we are in presence of **overfitting**.

In the next notebooks, we will see in practice how to reduce **overfitting**.

Now, we can take the opportunity to define what accuracy means:

In [ ]:
Y_train = model.embedding.dval.Y  # True labels (i.e. target values)
A_train = model.embedding.dval.A  # Probabilities - Output of model
P_train = model.embedding.dval.P  # Decision from probabilities

for y, a, p in list(zip(Y_train, A_train, P_train))[:10]:
    print(y, a, p)

This print denotes for each row, by column:

* The true label or target value: It is ``0`` or ``1`` herein.
* The output probability: The value is within \[0, 1\]. Therefore, the higher the probability the more confident the model is to predict a label of ``1``. Conversely, the lower the probability the more confident the model is to predict a label of ``0``.
* The predicted label or output decision: This is the rounding of the output probability to the nearest integer.

When the number of output nodes is equal to one, rounding of the output probability is as follows:

In [ ]:
print(all(np.around(A_train) == P_train))

Then, the accuracy for each sample is defined as:

In [ ]:
print((Y_train == P_train))

Note that the accuracy for each sample is `True` or `False` which evaluates to `1` and `0`, respectively.

To compute the accuracy with respect to the whole dataset, we need the mean:

In [ ]:
print((Y_train == P_train).mean())
print(np.around((Y_train == P_train).mean(), 3))

Note the rounded value is, as expected, identical to the accuracy reported for the last epoch on the validation set.

We can finally plot the results:

In [ ]:
model.plot(path=False)

In this plot, *dtrain* and *dval* refer to the training and validation set and accuracy/MSE values are identical to those printed on the terminal. This is simply a graphical representation of the tabular log report.

For code, maths and pictures behind the EpyNN model, follow this link:

* [Neural Network - Model](https://epynn.net/EpyNN_Model.html)

## Write, read & Predict

A trained model can be written on disk such as:

In [ ]:
model.write()

# model.write(path=/your/custom/path)

A model can be read from disk such as:

In [ ]:
model = read_model()

# model = read_model(path=/your/custom/path)

We can retrieve new features and predict on them.

In [ ]:
X_features, _ = prepare_dataset(N_SAMPLES=10)

dset = model.predict(X_features)

Results can be extracted such as:

In [ ]:
for n, pred, probs, features in zip(dset.ids, dset.P, dset.A, dset.X):
    print(n, pred, probs, features)
    # pred = output (decision); probs = output (probability)